#### 460. LFU Cache

* https://leetcode.com/problems/lfu-cache/description/

#### BBG - IMP

In [29]:
from collections import defaultdict
class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.frequency = 1
        self.prev = None
        self.next = Node


class DoublyLinkedList:
    def __init__(self):
        self.head = Node(None, None)
        self.tail = Node(None, None)

        self.head.next = self.tail
        self.tail.prev = self.head

        self.size = 0

    def add_to_front(self, node):
        node.next = self.head.next
        node.prev = self.head

        self.head.next.prev = node
        self.head.next = node

        self.size += 1

    def remove_node(self, node):
        next_node = node.next
        prev_node = node.prev

        prev_node.next = next_node
        next_node.prev = prev_node

        self.size -= 1

    def lru_evict(self):
        if self.size == 0:
            return None
        lru_node = self.tail.prev
        self.remove_node(lru_node)
        return lru_node

    def isempty(self):
        return self.size == 0



class LFUCache:

    def __init__(self, capacity: int):
        self.capacity = capacity
        self.current_size = 0

        self.cache = {} # key -> None
        self.freq_to_dll = defaultdict(DoublyLinkedList)

        self.min_freq = 0
        

    def _update_freq(self, node):
        old_freq = node.frequency
        self.freq_to_dll[old_freq].remove_node(node)

        if old_freq == self.min_freq and self.freq_to_dll[old_freq].isempty():
            self.min_freq += 1

        node.frequency += 1
        self.freq_to_dll[node.frequency].add_to_front(node)


    def get(self, key: int) -> int:
        node = self.cache.get(key)
        if not node:
            return -1
        
        self._update_freq(node)
        return node.value
        

    def put(self, key: int, value: int) -> None:
        if self.capacity == 0:
            return

        node = self.cache.get(key)
        if node:
            node.value = value
            self._update_freq(node)
            return

        if self.capacity == self.current_size:
            lfu_list = self.freq_to_dll[self.min_freq]
            evicted_node = lfu_list.lru_evict()
            del self.cache[evicted_node.key]
            self.current_size -= 1 

        new_node = Node(key, value)
        self.cache[key] = new_node
        self.freq_to_dll[1].add_to_front(new_node)
        self.min_freq = 1
        self.current_size += 1




# Your LFUCache object will be instantiated and called as such:
# obj = LFUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)




# Your LFUCache object will be instantiated and called as such:
obj = LFUCache(2)
obj.put(10,100)
obj.put(20,200)
print(obj.get(10))
obj.put(30,300)
print(obj.get(2))

100
-1


In [27]:
from collections import defaultdict


class Node:
    """
    Doubly linked list node.
    Each node represents a cache entry.
    """
    __slots__ = ("key", "value", "frequency", "prev", "next")

    def __init__(self, key: int, value: int):
        self.key = key
        self.value = value
        self.frequency = 1
        self.prev = None
        self.next = None


class DoublyLinkedList:
    """
    Maintains LRU order for nodes with the same frequency.
    New nodes are added to the front (most recently used).
    """
    def __init__(self):
        self.head = Node(None, None)
        self.tail = Node(None, None)
        self.head.next = self.tail
        self.tail.prev = self.head
        self.size = 0

    def add_to_front(self, node: Node) -> None:
        node.next = self.head.next
        node.prev = self.head
        self.head.next.prev = node
        self.head.next = node
        self.size += 1

    def remove_node(self, node: Node) -> None:
        node.prev.next = node.next
        node.next.prev = node.prev
        self.size -= 1

    def remove_lru(self) -> Node:
        """
        Remove least recently used node (from the end).
        """
        if self.size == 0:
            return None
        lru_node = self.tail.prev
        self.remove_node(lru_node)
        return lru_node

    def is_empty(self) -> bool:
        return self.size == 0


class LFUCache:
    """
    LFU Cache with O(1) get and put.
    """
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.current_size = 0
        self.min_frequency = 0

        self.key_to_node = {}
        self.freq_to_list = defaultdict(DoublyLinkedList)

    def _update_frequency(self, node: Node) -> None:
        """
        Moves a node from its current frequency list
        to the next higher frequency list.
        """
        old_freq = node.frequency
        self.freq_to_list[old_freq].remove_node(node)

        if old_freq == self.min_frequency and self.freq_to_list[old_freq].is_empty():
            self.min_frequency += 1

        node.frequency += 1
        self.freq_to_list[node.frequency].add_to_front(node)

    def get(self, key: int) -> int:
        if key not in self.key_to_node:
            return -1

        node = self.key_to_node[key]
        self._update_frequency(node)
        return node.value

    def put(self, key: int, value: int) -> None:
        if self.capacity == 0:
            return

        if key in self.key_to_node:
            node = self.key_to_node[key]
            node.value = value
            self._update_frequency(node)
            return

        if self.current_size == self.capacity:
            lfu_list = self.freq_to_list[self.min_frequency]
            evicted_node = lfu_list.remove_lru()
            del self.key_to_node[evicted_node.key]
            self.current_size -= 1

        new_node = Node(key, value)
        self.key_to_node[key] = new_node
        self.freq_to_list[1].add_to_front(new_node)
        self.min_frequency = 1
        self.current_size += 1


obj = LFUCache(2)
obj.put(10,100)
obj.put(20,200)
print(obj.get(10))
obj.put(30,300)
print(obj.get(20))


100
-1
